In [13]:
#ask for user id, playback client, and active device id
#for instructions on how to access your Spotify User ID please reference the appendix slides

def get_active_device(spotify_playback):
    #get user devices 
    devices = spotify_playback.devices()
    devices_df = pd.DataFrame.from_dict(devices, orient='index')
    devices = devices['devices']
    all_devices = pd.DataFrame()
    for i in range(0, len(devices)):
        devices_df = pd.DataFrame.from_dict(devices[i], orient='index')
        all_devices = all_devices.append(devices_df.T) 
    #get active device
    active_device_id = all_devices[all_devices['is_active'] == True]['id']
    return active_device_id.iloc[0]

In [15]:
#ask for playlist id
#for instructions on how to access your playlist ID please reference the appendix slides

def get_playlist(spotify_playback):
    #enter in playlist_id
    playlist_id=input('Please enter in the playlist id: ')
    track_uris = []
    #get track uris for first 100 songs in playlist
    tracks = spotify_playback.playlist_tracks(playlist_id, fields=None, limit=100, offset=0, market=None)
    for i in range(0, len(tracks['items'])):
        track_uris.append(tracks['items'][i]['track']['uri'])
    return playlist_id, track_uris

In [16]:
#ask user if they liked a song

def track_pref(spotify_streaming, track_uris, active_device_id, selected_tracks):
    #select a random song in playlist
    play_this = random.choice(track_uris)
    #remove this track from track_uris
    track_uris.remove(play_this)
    #play the randomly selected song
    spotify_streaming.start_playback(device_id=active_device_id, context_uri=None, uris=[play_this], offset=None, position_ms=None)
    pref = input("Do you enjoy this song? Y/N: ") 
    if pref.lower() == 'y':
        #keep track of the selected songs
        selected_tracks.append(play_this)
    return track_uris, selected_tracks

In [21]:
#ask user if they want to check out other songs

def next_track(spotify_streaming, track_uris, active_device_id, selected_tracks):
    again = input('Do you want to explore more songs? Y/N: ')
    while again.lower() == 'y':
        track_uris, selected_tracks = track_pref(spotify_streaming, track_uris, active_device_id, selected_tracks)
        again = input('Do you want to explore more songs? Y/N: ')
    return selected_tracks

In [17]:
#get selected tracks and the 3 closest songs 
#distance computed using select audio features and euclidean distance

def get_tracks(spotify_playback, selected_tracks):
    add_to_playlist = pd.DataFrame()
    #loop over selected tracks
    for i in range(0, len(selected_tracks)):
        #get recs for each selected track
        recs = spotify_playback.recommendations(seed_artists=None, seed_genres=None, seed_tracks=[selected_tracks[i]], limit=20, country=None)
        recs = recs['tracks']
        rec_uris = []
        for j in range(0, len(recs)):
            rec_uris.append(recs[j]['uri'])
        #get audio features for selected track
        track_features = spotify_playback.audio_features(selected_tracks[i])
        all_track_features = pd.DataFrame()
        for k in range(0, len(track_features)):
            selected_track_features = pd.DataFrame.from_dict(track_features[k], orient='index')
            selected_track_features = selected_track_features.T
            all_track_features = all_track_features.append(selected_track_features)
        #get audio features for recs
        track_features = spotify_playback.audio_features(rec_uris)
        for l in range(0, len(track_features)):
            single_rec_features = pd.DataFrame.from_dict(track_features[l], orient='index')
            single_rec_features = single_rec_features.T
            all_track_features = all_track_features.append(single_rec_features)
        #find 3 closest recs to selected_track
        subset_features = all_track_features.drop(['mode','type','key', 'id', 'uri', 'track_href','analysis_url', 'duration_ms','time_signature'], axis=1)
        #4 because distance between selected track and itself is 0 and we want to add the selected track and 3 closest neighbors to the playlist
        idx = np.argpartition(np.array(euclidean_distances(subset_features)[0]), 4)
        add_to_playlist = add_to_playlist.append(all_track_features.iloc[idx[:4]])
    return add_to_playlist


In [19]:
#create final playlist of selected songs and closest 3 recs

def create_playlist(spotify_playlist, add_to_playlist):
    #ask user for name for playlist
    playlist_name = input('Enter a name for your playlist: ')
    playlist_json = spotify_playlist.user_playlist_create(user_id, playlist_name)
    spotify_playlist.user_playlist_add_tracks(user_id, playlist_id=playlist_json['id'], tracks=add_to_playlist['uri'])
    return playlist_name

In [20]:
#import packages
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import random
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

#main function to execute whole pipeline
#when new tab appears simply copy the url and paste into the dialogue box below -- this will be done 3 times
#after copying the url you can close the new tab

def main():
    SPOTIFY_CLIENT_ID='f6d1c9cfff924b22ac007510da646114'
    SPOTIFY_CLIENT_SECRET='5197e1f6f7f643748fdf99d7d1d39f51'
    SPOTIFY_REDIRECT_URI='http://localhost:8888/callback'
    #get user id
    user_id = input("Please enter your Spotify User ID: ")
    #prompt user to start playing track 
    playing = input('Please start playing a song on your desired device. Press Enter once a song is playing:')
    if playing == "":
        #authenticate for playback 
        scope ='user-read-playback-state'
        username = user_id
        token_playback = util.prompt_for_user_token(username, scope, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI)
        spotify_playback = spotipy.Spotify(auth=token_playback)
        #authenitcate for playlist modification
        scope = 'playlist-modify-public'
        token_playlist = util.prompt_for_user_token(username, scope, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI)
        spotify_playlist = spotipy.Spotify(auth=token_playlist)
        #authenticate for streaming
        scope = 'streaming'
        token_streaming = util.prompt_for_user_token(username, scope, SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI)
        spotify_streaming = spotipy.Spotify(auth=token_streaming)
        #init selected tracks as empty list
        selected_tracks = []
        #run functions
        active_device_id = get_active_device(spotify_playback)
        playlist_id, track_uris = get_playlist(spotify_playback)
        track_uris, selected_tracks = track_pref(spotify_streaming, track_uris, active_device_id, selected_tracks)
        selected_tracks = next_track(spotify_streaming, track_uris, active_device_id, selected_tracks)
        add_to_playlist = get_tracks(spotify_playback, selected_tracks)
        playlist_name = create_playlist(spotify_playlist, add_to_playlist)
        print('Thanks you for your input. Your playlist titled ' + playlist_name + ' is ready for listening!')
    return True

main()

#user_id 1210528719
#playlist_id 37i9dQZF1E36NxUPPWhYab

Please enter your Spotify User ID: 1210528719
Please start playing a song on your desired device. Press Enter once a song is playing:


            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=f6d1c9cfff924b22ac007510da646114&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-read-playback-state in your browser


Enter the URL you were redirected to: http://localhost:8888/callback?code=AQAdzc1OdfSsXSqA0Imyli6Gk8PvXqS9V52OfVUC3jvAvbPi3gJZhw4bJbNYpGbhdhTF-s6vxX96iRSKsx7LkTRg-8X0qS4CeLqE7ary7kVoa7APt8crxPW6Iu9_XVj3-NLMx3MSjO06PYbEmbzD20wQvnp5swXPnq27XpYAaZBwdR9MvHozNM9t2RE9wZRMliE_i4knR2wCNyayysNZAbijW-m3MWF1ZV9ZuKd8




            User authentication requires interactio

True